In [1]:
import os
import whisper
import yt_dlp
from openai import OpenAI

def download_audio(url, output_path="audio"):
    try:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        ydl_opts = {
            'format': 'm4a/bestaudio/best',
            'paths': {'home': output_path},
            'outtmpl': {'default': '%(title)s.%(ext)s'},
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
            }],
            'quiet': False,
            'no_warnings': True
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print("Downloading audio...")
            info = ydl.extract_info(url, download=True)
            audio_file = os.path.join(output_path, f"{info['title']}.mp3")
            return audio_file
            
    except Exception as e:
        print(f"Error downloading audio: {str(e)}")
        return None

def transcribe_audio(audio_path, model_size="base"):
    try:
        print("Loading Whisper model...")
        model = whisper.load_model(model_size)
        
        print("Starting transcription...")
        result = model.transcribe(audio_path)
        
        return result["text"]
        
    except Exception as e:
        print(f"Error during transcription: {str(e)}")
        return None

def summarize_text(transcription_text, openai_api_key):
    try:
        client = OpenAI(api_key=openai_api_key)
        
        print("Generating summary...")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant that summarizes long texts."},
                {"role": "user", "content": f"Summarize the following content: {transcription_text}. Give 10 very important bullet points"}
            ]
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"Error generating summary: {str(e)}")
        return None

def process_video(url, openai_api_key, model_size="base", output_path="audio", save_audio=False):
    """
    Process video: download, transcribe, and summarize
    """
    audio_file = download_audio(url, output_path)
    if not audio_file:
        return None

    transcription = transcribe_audio(audio_file, model_size)
    if not transcription:
        return None

    summary = summarize_text(transcription, openai_api_key)

    if not save_audio:
        try:
            os.remove(audio_file)
            print("Audio file deleted")
        except:
            print("Warning: Could not delete audio file")
    else:
        print(f"Audio file saved: {audio_file}")

    return summary

def get_video_summary(video_url, api_key, save_audio=False):
    """
    Main function to get video summary
    """
    if not video_url or not api_key:
        raise ValueError("Both video URL and API key are required")

    summary = process_video(
        video_url,
        api_key,
        save_audio=save_audio
    )

    if summary:
        with open("summary.txt", "w", encoding="utf-8") as f:
            f.write(summary)
        print("Summary saved to summary.txt")
        return summary
    return None

# Example usage:
summary = get_video_summary("link", "API", save_audio=False)
if summary:
    print(summary)

[youtube] Extracting URL: https://youtu.be/Mlv6GfjzvPQ?si=E_G92-Mr-FCDXhMm
[youtube] Mlv6GfjzvPQ: Downloading webpage
[youtube] Mlv6GfjzvPQ: Downloading tv player API JSON
[youtube] Mlv6GfjzvPQ: Downloading ios player API JSON
[youtube] Mlv6GfjzvPQ: Downloading m3u8 information
[info] Mlv6GfjzvPQ: Downloading 1 format(s): 140
[download] Destination: audio\DeSantis Interrupts Reporter He Says Is Conflating Legal And Illegal Immigrants.m4a
[download] 100% of    4.33MiB in 00:00:00 at 4.46MiB/s     
[FixupM4a] Correcting container of "audio\DeSantis Interrupts Reporter He Says Is Conflating Legal And Illegal Immigrants.m4a"
[ExtractAudio] Destination: audio\DeSantis Interrupts Reporter He Says Is Conflating Legal And Illegal Immigrants.mp3
Deleting original file audio\DeSantis Interrupts Reporter He Says Is Conflating Legal And Illegal Immigrants.m4a (pass -k to keep)
Loading Whisper model...


c:\Users\alokd\miniconda3\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Starting transcription...


c:\Users\alokd\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Generating summary...
Audio file deleted
Summary saved to summary.txt
1. Legal immigrants make up a significant portion of the workforce in industries like hospitality and tourism in Florida.
2. Concerns exist about certain legal immigration programs, such as the H1B program, where Americans may train H1B workers only to be replaced by them.
3. Chain migration is criticized for bringing in more distant relatives without merit-based processes.
4. Proper enforcement distinguishes between legal immigrants, like those on H2A visas, and illegal aliens.
5. Calls for reform in legal immigration processes are made to prioritize American interests.
6. The first generation of immigrants supports immigration enforcement, as seen in election results in Miami-Dade County.
7. Differentiation is emphasized between illegal aliens and legal immigrants in discussions of immigration enforcement.
8. Collaboration is expected between Florida and the Trump administration to achieve objectives, with more sta